<a href="https://colab.research.google.com/github/Athugodage/RuLawSimplification/blob/main/gpt_to_simplify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Athugodage/RuLawSimplification.git

Cloning into 'RuLawSimplification'...
remote: Enumerating objects: 435, done.
remote: Counting objects: 100% (170/170), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 435 (delta 84), reused 152 (delta 79), pack-reused 265
Receiving objects: 100% (435/435), 243.13 MiB | 49.92 MiB/s, done.
Resolving deltas: 100% (197/197), done.


In [2]:
!unzip '/content/RuLawSimplification/corpus/cut_corpus.zip'

Archive:  /content/RuLawSimplification/corpus/cut_corpus.zip
  inflating: cut_corpus.csv          


In [15]:
import pandas as pd

df = pd.read_csv('cut_corpus.csv')

In [16]:

df = df.dropna()

In [17]:
df

,Cut_original,Cut_comment
0,"При этом должны использоваться не чрезмерные,...","Бессрочный запрет на профессию для адвокатов,..."
1,"Суд, установив факт нарушения прав потребител...",Пленум Верховного суда России утвердил постан...
2,"В состав семьи, учитываемый при определении п...",Единое пособие для беременных женщин и гражда...
4,Перечисление средств по таким контрактам (дог...,Приостановленное в этом году бюджетное правил...
5,Принят Государственной Думой 8 ноября 2022 го...,"""Российская газета"" публикует закон № 438-ФЗ ..."
...,...,...
1980,Правила по проведению работ в системе сертифи...,Через десять дней автомобиль можно будет пост...
1981,Правительство Российской Федерации постановля...,Правительство утвердило изменения в правила н...
1982,"Установить, что федеральные государственные у...","Правительство приняло постановление, расширяю..."
1983,Утвердить прилагаемые требования к транспортн...,Специфическим образом теперь должны быть раск...


In [18]:
from sklearn.model_selection import train_test_split

In [19]:
train, test = train_test_split(df, random_state=42, test_size=0.05)

In [20]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
!pip install sacremoses
!pip install sacrebleu
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=b6e3864359fafa8c2eb773f2238831d19c904f727d5f8c239c6ab27c5b2193bb
  Stored in directory: /root/.cache/pip/wheels/12/1c/3d/46cf06718d63a32ff798a89594b61e7f345ab6b36d909ce033
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 4.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 98.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, 

In [ ]:
!nvidia-smi

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
train.columns = ["text", "commentary"]
test.columns = ["text", "commentary"]

In [13]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [14]:
import torch
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer
from datasets import Dataset



def tokenize_function(examples):
    law_tokens = tokenizer(examples["text"], truncation=True, max_length=512, padding='max_length', return_special_tokens_mask=True)
    special_tok = tokenizer(" [SEP] ", return_special_tokens_mask=True)
    commentary_tokens = tokenizer(examples["commentary"], truncation=True, max_length=512, padding='max_length', return_special_tokens_mask=True)
    tokens = {key: torch.cat((torch.tensor(law_tokens[key]), torch.tensor(special_tok[key]), torch.tensor(commentary_tokens[key])), dim=-1) for key in law_tokens}
    return tokens

# Convert pandas DataFrames to Hugging Face Dataset objects
train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)
# Tokenize the train and validation datasets
ds = train_dataset.map(tokenize_function, remove_columns=['text', 'commentary', '__index_level_0__'])
val = test_dataset.map(tokenize_function, remove_columns=['text', 'commentary', '__index_level_0__'])

# Set up the data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)



Map:   0%|          | 0/1755 [00:00<?, ? examples/s]

Map:   0%|          | 0/93 [00:00<?, ? examples/s]

In [21]:
from huggingface_hub import notebook_login

notebook_login()
     

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [22]:
from datasets import load_metric


sari = load_metric("sari")

<ipython-input-22-46b18dc41a4f>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  sari = load_metric("sari")


In [23]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

In [24]:
def compute_metrics(eval_preds):
    inputs, preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    if data_args.ignore_pad_token_for_loss:
        # Replace -100 in the labels as we can't decode them.
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        inputs = np.where(inputs != -100, inputs, tokenizer.pad_token_id)

    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_inputs = tokenizer.batch_decode(inputs, skip_special_tokens=True)

    # Some simple post-processing
    decoded_inputs, decoded_preds, decoded_labels = postprocess_text(decoded_inputs, decoded_preds,
                                                                        decoded_labels, "sari")
    sari_result = sari._compute(inputs=decoded_inputs, predictions=decoded_preds, references=decoded_labels)

In [28]:
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")


training_args = TrainingArguments(
    output_dir="GPT_simplifier1800",
    num_train_epochs=7,
    per_device_train_batch_size=3,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="logs",
    push_to_hub=True
)

# Set up the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds,
    eval_dataset=val,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()


/usr/local/lib/python3.9/dist-packages/transformers/models/auto/modeling_auto.py:1295: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
/content/GPT_simplifier1800 is already a clone of https://huggingface.co/marcus2000/GPT_simplifier1800. Make sure you pull the latest changes with `repo.git_pull()`.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


OutOfMemoryError: ignored

In [ ]:
from transformers import pipeline

In [ ]:
test.iloc[0]["text"][:2400] + " [SEP] "

In [ ]:
device = model.device

In [ ]:
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer,device=0)

# Generate text
prompt = test.iloc[0]["text"][:2400] + " [SEP] "
generated_text = text_generator(prompt, max_length=1800, num_return_sequences=1)

In [ ]:

generated_text